# Project Setup Notebook

This notebook bootstraps IDs needed by the project. It will:
1) Load your environment from `.env`.
2) Create an aiXplain **Index** if `INDEX_ID` is missing.
3) Optionally verify an existing **LLM_ID** and **Agent** (if you choose).
4) Persist the new `INDEX_ID` back into `.env`.

**Requirements**: a valid `AIXPLAIN_API_KEY` in `.env`.

In [ ]:
import sys, subprocess
def pip_install(pkgs):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q'] + pkgs)
pip_install(['aixplain', 'python-dotenv'])
print('Dependencies ready')

In [ ]:
import os
from dotenv import load_dotenv
from pathlib import Path
from datetime import datetime

ENV_PATH = Path('.env')
load_dotenv(dotenv_path=ENV_PATH if ENV_PATH.exists() else None)

def write_env_var(key: str, value: str, env_path: Path = ENV_PATH):
    lines = []
    if env_path.exists():
        with env_path.open('r', encoding='utf-8') as f:
            lines = f.read().splitlines()
    found = False
    out = []
    for line in lines:
        if line.strip().startswith(f'{key}='):
            out.append(f'{key}={value}')
            found = True
        else:
            out.append(line)
    if not found:
        out.append(f'{key}={value}')
    with env_path.open('w', encoding='utf-8') as f:
        f.write('\n'.join(out) + ('\n' if out and out[-1] else ''))
    os.environ[key] = value
    return value

api_key = os.getenv('AIXPLAIN_API_KEY')
if not api_key:
    raise RuntimeError("AIXPLAIN_API_KEY is missing in .env — add it and rerun this cell.")
print('Loaded AIXPLAIN_API_KEY ✅')

In [ ]:
from aixplain.factories import IndexFactory
import os
from datetime import datetime

INDEX_ID = os.getenv('INDEX_ID')
if INDEX_ID:
    try:
        idx = IndexFactory.get(INDEX_ID)
        _ = getattr(idx, 'id', None)
        print('Found existing INDEX_ID:', INDEX_ID)
    except Exception as e:
        print('Existing INDEX_ID invalid, creating a new index...')
        idx = IndexFactory.create(
            name=f'PolicyNavigatorIndex-{datetime.utcnow().strftime("%Y%m%d-%H%M%S")}',
            description='Index for Policy Navigator project (auto-created by setup notebook).'
        )
        INDEX_ID = idx.id
        write_env_var('INDEX_ID', INDEX_ID)
        print('Created and saved new INDEX_ID:', INDEX_ID)
else:
    idx = IndexFactory.create(
        name=f'PolicyNavigatorIndex-{datetime.utcnow().strftime("%Y%m%d-%H%M%S")}',
        description='Index for Policy Navigator project (auto-created by setup notebook).'
    )
    INDEX_ID = idx.id
    write_env_var('INDEX_ID', INDEX_ID)
    print('Created and saved new INDEX_ID:', INDEX_ID)

INDEX_ID

### Optional: Verify LLM and create a simple Agent (skips if `LLM_ID` is missing)
This is not required for indexing, but useful for quick end-to-end checks.

In [ ]:
import os
from datetime import datetime
from aixplain.factories import AgentFactory

LLM_ID = os.getenv('LLM_ID')
if not LLM_ID:
    print('LLM_ID not set — skipping agent creation.')
else:
    try:
        agent = AgentFactory.create(
            name=f'PolicyNavigator-Agent-{datetime.utcnow().strftime("%Y%m%d-%H%M%S")}',
            description='Simple agent for quick tests created by setup notebook.',
            instructions='Answer briefly and include citations when possible.',
            tools=[],
            llm_id=LLM_ID,
        )
        print('Agent created. ID:', agent.id)
    except Exception as e:
        print('Agent creation failed (likely invalid LLM_ID or permissions):', e)

### Summary
- `INDEX_ID` is now guaranteed in `.env`.
- If you provided `LLM_ID`, a temporary agent may have been created for smoke testing.

Next steps:
1) Run your ingestion (`ingest`/`indexer`) to add data into the index.
2) Run your pipeline/agent code — they will read `INDEX_ID` from `.env`.